In [26]:
import warnings

import pandas as pd
import numpy as np
import torch

from MLWrappers import SklearnBlackBox
from PrivacyAttacks import MiaPrivacyAttack, LabelOnlyPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [27]:
MODEL_NAME = 'rf'
DS_NAME = 'adult'
DATA_FOLDER = f'./data/{DS_NAME}'

# We load the data used to train and test the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)
train_label = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_label.csv', skipinitialspace=True)
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)


In [28]:
# we load the target black box model using our wrapper
target = SklearnBlackBox(f'./models/{DS_NAME}_{MODEL_NAME}.pkl')

In [29]:
# We can define the parameters to be passed to the shadow models (in our case random forests)
shadow_forest_params = {'n_estimators': 100, 'max_depth': 5}

In [30]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target,
                       n_shadow_models=5)
label_only = LabelOnlyPrivacyAttack(target,
                                    n_shadow_models=1,
                                    shadow_model_type='rf',
                                    shadow_model_params=shadow_forest_params,
                                    n_noise_samples_fit=1000,
                                    shadow_test_size=0.45)
aloa = AloaPrivacyAttack(target,
                         n_shadow_models=1,
                         shadow_model_type='rf',
                         shadow_model_params=shadow_forest_params,
                         n_noise_samples_fit=1000,
                         shadow_test_size=0.45,
                         undersample_attack_dataset=True)
attacks = [
    mia,
    label_only,
    aloa
]

In [31]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the list of attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [32]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set, save_folder = f'./results_{DS_NAME}_{MODEL_NAME}')

100%|██████████| 5432/5432 [02:20<00:00, 38.53it/s]


In [33]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)

  7%|▋         | 1677/24134 [00:46<10:29, 35.68it/s]


KeyboardInterrupt: 

In [34]:
results['mia_attack']['classification_report']

{'IN': {'precision': 0.8405278592375367,
  'recall': 0.7422696431346144,
  'f1-score': 0.7883488736694446,
  'support': 19307},
 'OUT': {'precision': 0.29757199322416716,
  'recall': 0.436710171949451,
  'f1-score': 0.3539585257325161,
  'support': 4827},
 'accuracy': 0.6811552167067209,
 'macro avg': {'precision': 0.5690499262308519,
  'recall': 0.5894899075420327,
  'f1-score': 0.5711536997009804,
  'support': 24134},
 'weighted avg': {'precision': 0.73193218652491,
  'recall': 0.6811552167067209,
  'f1-score': 0.701467204261491,
  'support': 24134}}

In [37]:
results['label_only_attack']['classification_report']

{'IN': {'precision': 0.7999917129361067,
  'recall': 1.0,
  'f1-score': 0.8888837733937984,
  'support': 19307},
 'OUT': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4827},
 'accuracy': 0.7999917129361067,
 'macro avg': {'precision': 0.39999585646805336,
  'recall': 0.5,
  'f1-score': 0.4444418866968992,
  'support': 24134},
 'weighted avg': {'precision': 0.6399867407664462,
  'recall': 0.7999917129361067,
  'f1-score': 0.7110996524784149,
  'support': 24134}}

In [38]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.7995006242197253,
  'recall': 0.9950795048428032,
  'f1-score': 0.8866326695433464,
  'support': 19307},
 'OUT': {'precision': 0.08653846153846154,
  'recall': 0.0018645121193287756,
  'f1-score': 0.003650375177448793,
  'support': 4827},
 'accuracy': 0.7964282754620038,
 'macro avg': {'precision': 0.44301954287909345,
  'recall': 0.498472008481066,
  'f1-score': 0.4451415223603976,
  'support': 24134},
 'weighted avg': {'precision': 0.656902283320477,
  'recall': 0.7964282754620038,
  'f1-score': 0.7100288933394769,
  'support': 24134}}